# Convolution using FPGAs

When targeting FPGAs, OpenCL kernels have a lot of freedom in choosing the hardware organization of the design. An FPGA is a blank slate -- subject to logic/memory capacity limits, it can be configured as you desire to (1) implement the kernel is pipelined, dataflow fashion for high throughput, (2) program RAM hierarchy, access layout, port organization, data-widths to match application needs, and (3) populate the FPGA with as many tiles as feasible. 

*With great power comes great responsibility.*

![](spiderman.jpg)

We will explore the three different axes of freedom when mapping OpenCL kernels to FPGA using Xilinx SDAccel on the AlphaData PCIe card.

## 1. Kernel Optimizations
We can think of each OpenCL kernel as mapping to its own datapath on the FPGA. This datapath is analogous to an OpenCL **Processing Element** inside an OpenCL **Compute Unit**. Unlike traditional OpenCL perspective, you only have a single datapath/PE inside the **Compute Unit**. However, thorugh optimizations such as **kernel workitem pipelining**, we expose multiple pipeline slots in the datapath to the different work items. It is useful to think of each of these pipeline slots as a **Processing Element**. We show how to optimize the 2D convolution kernel using three OpenCL hints as shown below.  

In [ ]:

__kernel __attribute__ ((reqd_work_group_size(W, H, 1)))
void conv_2d(
    __global float *in,               // W*H input images
    __constant float *filt,           // K*K filter kernel
    __global float *out,              // W*H output images
    const float pBias)                // constant offset/bias
{
    __local float local_image[W * H];
    __local float local_filt[K * K];
    int x = get_local_id(0);
    int y = get_local_id(1);
    
    __attribute__((xcl_pipeline_workitems)) {
        if(x < FILTER_SIZE*FILTER_SIZE) {
            local_filt[x] = filt[x];
        }
        local_image[y*H + x] = in[y*H + x];
    }
    // wait for all loads to complete
    barrier(CLK_LOCAL_MEM_FENCE);

    float sum = 0;
    // pipeline the implicit loop over pixels
    __attribute__((xcl_pipeline_workitems)) {
        __attribute__((opencl_unroll_hint))
        for (int r = 0; r < K; r++) {
            __attribute__((opencl_unroll_hint))
            for(int c = 0; c < K; c++){
                sum += local_filt[r*K + c]*local_image[(y+r)*W + x + c];
            }
        }
        out[y*H + i] = sum + pBias;
    }
}

In the code block above, we notice the following three big ideas.

1. The use of **__local** memory storage for accessing pixels and the input image. This is useful to ensure we can batch-load the data from the off-chip device RAM into the on-chip FPGA Block RAMs for fast access. For deep learning applications, the image maps tend to be small, and can easily fit inside the on-chip capacities for many cases. However, the storage of kernel weights if often a bottleneck, and this is something to watch out for later. Unlike GPUs with ~64KB local RAM capacity per SM (and ~dozens of SMs), a large modern FPGA can support 3--4MB of on-chip storage that can be configured as local RAMs.
2. We also use the **xcl_pipeline_workitems** OpenCL hint that is Xilinx-specific compiler input. It tells the compiler to pipeline the generated datapath across the multiple work items. The workgroup size (number of workitems) is specified earlier using **reqd_work_group_size** attribute. While this improves performance, the scheduling of work items in this fashion can impose a significant memory bandwidth requirement, while also adding to FPGA resource cost (number of FFs).
3. Finally, the computations on each pixel require fetching the kernel neighbourhood. We can unroll these loops and achieve a throughput of one pixel per cycle. This adds to resource cost, i.e. more LUTs and DSPs to perform the multiply-accumulate operations in parallel. This also imposes some memory layout expectations on the data stored in RAMs.

## 2. Memory Layout Optimizations
Some of the OpenCL kernel optimizations improve the performance of the FPGA datapath, but expect the local memories to be organized in a specific fashion. On CPUs and GPUs, the caches, and shared RAMs (GPU-only) are fixed capacity, and are banked in fixed ways. It is the application's job to match these properties of the OpenCL device. On the FPGA, there is freedom in both these dimension. While, there is still an upper bound on total FPGA BRAM capacity, small mismatches in capacities can be managed. 
Here we explore the partitioning strategies for data layout in the local RAMs.



## 3. Compute Units
Finally, after all kernel and memory customziations, if we have a large-enough FPGA chip, we can use *tiling* optimization that replicates the **Compute Unit** as many times as feasible. Recall, the OpenCL kernel maps to an FPGA datapath which is a **Compute Unit** by itself. The specification for concurrency of compute units is supplied in a *tcl* script that drives the SDAccel compiler. This *tcl* script can be thought of as *gcc* Makefile for an FPGA.

In [ ]:
# Setup the FPGA target
create_solution -name sol_1 -dir . -force
add_device -vbnv xilinx:adm-pcie-7v3:1ddr:3.0
set host_args "conv2d.xclbin input.pgm"
set_property -name host_cflags -value "-g -O0 -std=c++0x -I$::env(PWD)" -objects [current_solution]

# Add files
add_files "host_app.c"
add_files "pgm.h"
set_property file_type "c header files" [get_files "pgm.h"]

# Kernel Definition
set ker_name conv2d
create_kernel $ker_name -type clc
add_files -kernel [get_kernels $ker_name] "kernel.cl"

# Define Binary Containers
create_opencl_binary $ker_name
set_property region "OCL_REGION_0" [get_opencl_binary $ker_name]
create_compute_unit -opencl_binary [get_opencl_binary $ker_name] 
                    -kernel [get_kernels $ker_name] -name inst0
create_compute_unit -opencl_binary [get_opencl_binary $ker_name] 
                    -kernel [get_kernels $ker_name] -name inst1

    
# Compile the design for CPU based emulation
compile_emulation -flow cpu
puts "Comipled for CPU emulation..."
run_emulation -flow cpu -args $host_args

# Create estimated resource usage and latency report
report_estimate

# Compile the design for Hardware Emulation
compile_emulation -flow hardware
run_emulation -flow hardware -args $args

# Compile the design for execution on the FPGA board
build_system

# Create the board deployment package for the application
package_system

In the *tcl* script above, the command **create_compute_unit** needs to be replicated as many times as desired to generate multiple **Compute Units** on the FPGA. Of course, we must ensure the global data is split across these compute units along some natural dimension of partitioning.